In [3]:
from http import client
from google.cloud import bigquery
from google.cloud.bigquery import SchemaField
import pandas as pd
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "./gtm-ops-9e9a928ef0ac.json"
client = bigquery.Client() 

In [5]:
# Path to  merged file
file_path = 'Data_Downloads/archive/march24/merged_files.csv'
df = pd.read_csv(file_path)
df_info = df.columns.tolist()

print(df_info)

['first_name', 'last_name', 'Job_Title', 'Job_Function', 'Email_Address', 'Email_Domain', 'LinkedIn_Contact_Profile_URL', 'Country', 'State', 'City', 'Company_Name', 'Website', 'Revenue_Range', 'Primary_Industry', 'LinkedIn_Company_Profile_URL']


In [7]:
# Define the main table where you want to load the data
table_id_main = "gtm-ops.profile_lists.main_table"

# Load the CSV data directly to the main table
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,  # Assuming the first row is header
    autodetect=False,  # Automatically detect schema (use with caution)
    write_disposition="WRITE_APPEND",  # Append to existing table
)
with open(file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id_main, job_config=job_config)
job.result()  # Wait for the job to complete
    

LoadJob<project=gtm-ops, location=europe-north1, id=3529a905-6379-4e69-857e-fbde8dfebb8b>

# Utilities

### Remove empty email rows

In [10]:
# Construct the query to remove rows without an email
delete_query = f"""
DELETE FROM `{table_id_main}`
WHERE Email_Address IS NULL OR TRIM(Email_Address) = ''
"""

# Execute the query
client.query(delete_query).result()


### Merge two tables avoiding duplicates

In [ ]:
# Define the source and destination table IDs
source_table_id = "gtm-ops.profile_lists.consolidated_feb_2024"
destination_table_id = "gtm-ops.profile_lists.main_table"

# Construct the query to insert non-duplicate records
# Assuming 'Email_Address' is used as the unique identifier for duplicates
insert_query = f"""
INSERT INTO `{destination_table_id}` (SELECT * FROM `{source_table_id}` src
WHERE NOT EXISTS (
    SELECT 1 FROM `{destination_table_id}` dest
    WHERE dest.Email_Address = src.Email_Address
))
"""

# Execute the query
job = client.query(insert_query)
job.result()  # Wait for the job to complete

# Print the number of rows inserted
print(f"Number of rows inserted: {job.num_dml_affected_rows}")
